In [1]:
from utils import Logger,adjust_learning_rate,CreateLogger,create_cosine_learning_schedule,encode_test_label,set_seed
from model import Resnet_with_uncertainty_derm#不确定度
from dependency import *
from torch import optim
from torchcontrib.optim import SWA
from dataloader import generate_dataloader
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error #

In [2]:
def train(net, train_dataloader, model_name):
    net.set_mode("train")
    train_loss = 0
    train_dia_acc = 0
    train_sps_acc = 0
    for index,(clinic_image,derm_image,meta_data,label) in enumerate(train_dataloader):
        opt.zero_grad()
        
        clinic_image = clinic_image.cuda()
        derm_image = derm_image.cuda()
        #meta_data = meta_data.cuda()
        
        #Diagnostic label
        diagnosis_label = label[0].long().cuda()
        #Seven-Point Checklist labels
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        #uncertainty
        
        [(logit_diagnosis_derm,logit_pn_derm,logit_str_derm,logit_pig_derm,logit_rs_derm,logit_dag_derm,logit_bwv_derm,logit_vs_derm,logit_uncertainty_derm)] = net((derm_image))
        
        #derm
        prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_derm),dim = 1)
        
        prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_derm),dim = 1)

        prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_derm),dim = 1)

        prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_derm), dim = 1)

        prediction5 =torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_derm), dim = 1)

        prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_derm), dim = 1)

        prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_derm), dim = 1)

        prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_derm), dim = 1)

        diagnosis_label_t = diagnosis_label.unsqueeze(0)
        pn_label_t = pn_label.unsqueeze(0)
        str_label_t = str_label.unsqueeze(0)
        pig_label_t = pig_label.unsqueeze(0)
        rs_label_t = rs_label.unsqueeze(0)
        dag_label_t = dag_label.unsqueeze(0)
        bwv_label_t = bwv_label.unsqueeze(0)
        vs_label_t = vs_label.unsqueeze(0)
            
        #
        pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
        str_label_t = np.array(str_label_t.cpu(),dtype="int")
        pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
        rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
        dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
        bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
        vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
        diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")

        t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
        t = np.hstack((t,str_label_t.T))
        t = np.hstack((t,pig_label_t.T))
        t = np.hstack((t,rs_label_t.T))
        t = np.hstack((t,dag_label_t.T))
        t = np.hstack((t,bwv_label_t.T))
        t = np.hstack((t,vs_label_t.T))
        t = torch.tensor(t)
            
        prediction1 = prediction1.unsqueeze(0)
        prediction2 = prediction2.unsqueeze(0)
        prediction3 = prediction3.unsqueeze(0)
        prediction4 = prediction4.unsqueeze(0)
        prediction5 = prediction5.unsqueeze(0)
        prediction6 = prediction6.unsqueeze(0)
        prediction7 = prediction7.unsqueeze(0)
        prediction8 = prediction8.unsqueeze(0)
        prediction1 = np.array(prediction1.cpu(),dtype="int")
        prediction2 = np.array(prediction2.cpu(),dtype="int")
        prediction3 = np.array(prediction3.cpu(),dtype="int")
        prediction4 = np.array(prediction4.cpu(),dtype="int")
        prediction5 = np.array(prediction5.cpu(),dtype="int")
        prediction6 = np.array(prediction6.cpu(),dtype="int")
        prediction7 = np.array(prediction7.cpu(),dtype="int")
        prediction8 = np.array(prediction8.cpu(),dtype="int")
        
        p = np.hstack((prediction1.T,prediction2.T))
        p = np.hstack((p,prediction3.T))
        p = np.hstack((p,prediction4.T))
        p = np.hstack((p,prediction5.T))
        p = np.hstack((p,prediction6.T))
        p = np.hstack((p,prediction7.T))
        p = np.hstack((p,prediction8.T))
        p = torch.tensor(p)
        
        certainty_b = (t == p).long()
        certainty_ones = torch.tensor(np.ones(certainty_b.size()))
        uncertainty_label_derm = []
        for i in range(certainty_ones.size(0)):
            mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
            if mean_squared_error_tmp >=0.6:
                mean_squared_error_tmp = 1
            else:
                mean_squared_error_tmp = 0
            uncertainty_label_derm.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
        uncertainty_label_derm = torch.tensor(uncertainty_label_derm)
        #uncertainty
        uncertainty_label_derm = uncertainty_label_derm.float().cuda()
        
        loss_derm = torch.div(
          net.criterion(logit_diagnosis_derm, diagnosis_label)
        + net.criterion(logit_pn_derm, pn_label)
        + net.criterion(logit_str_derm, str_label)
        + net.criterion(logit_pig_derm, pig_label)
        + net.criterion(logit_rs_derm, rs_label)
        + net.criterion(logit_dag_derm, dag_label)
        + net.criterion(logit_bwv_derm, bwv_label)
        + net.criterion(logit_vs_derm, vs_label)
        + net.criterion_MSE(logit_uncertainty_derm,uncertainty_label_derm), 9)
        
        loss = loss_derm 
        
        dia_acc_all = torch.div(net.metric(logit_diagnosis_derm,diagnosis_label),int(derm_image.size(0)))
        sps_acc_all = torch.div(  net.metric(logit_pn_derm,pn_label)
                                + net.metric(logit_str_derm,str_label)
                                + net.metric(logit_pig_derm,pig_label)
                                + net.metric(logit_rs_derm,rs_label)
                                + net.metric(logit_dag_derm,dag_label)
                                + net.metric(logit_bwv_derm,bwv_label)
                                + net.metric(logit_vs_derm,vs_label),int(7 * derm_image.size(0)))
        loss.backward()
        opt.step()

        train_loss += loss.item()
        train_dia_acc += dia_acc_all.item()
        train_sps_acc += sps_acc_all.item()

    train_loss = train_loss / (index + 1) # Because the index start with the value 0f zero
    train_dia_acc = train_dia_acc / (index + 1)
    train_sps_acc = train_sps_acc / (index + 1)

    return train_loss,train_dia_acc,train_sps_acc

In [3]:
def validation(net,val_dataloader,model_name):
    net.set_mode('valid')
    val_loss = 0
    val_dia_acc = 0
    vaL_sps_acc = 0
    for index, (clinic_image, derm_image, meta_data, label) in enumerate(val_dataloader):

        clinic_image = clinic_image.cuda()
        derm_image   = derm_image.cuda()
        #meta_data    = meta_data.cuda()

        diagnosis_label = label[0].long().cuda()
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        

        with torch.no_grad():

            [(logit_diagnosis_derm,logit_pn_derm,logit_str_derm,logit_pig_derm,logit_rs_derm,logit_dag_derm,logit_bwv_derm,logit_vs_derm,logit_uncertainty_derm)] = net((derm_image))
        
            diagnosis_label_t = diagnosis_label.unsqueeze(0)
            pn_label_t = pn_label.unsqueeze(0)
            str_label_t = str_label.unsqueeze(0)
            pig_label_t = pig_label.unsqueeze(0)
            rs_label_t = rs_label.unsqueeze(0)
            dag_label_t = dag_label.unsqueeze(0)
            bwv_label_t = bwv_label.unsqueeze(0)
            vs_label_t = vs_label.unsqueeze(0)
        
            #
            pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
            str_label_t = np.array(str_label_t.cpu(),dtype="int")
            pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
            rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
            dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
            bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
            vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
            diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")
        
            t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
            t = np.hstack((t,str_label_t.T))
            t = np.hstack((t,pig_label_t.T))
            t = np.hstack((t,rs_label_t.T))
            t = np.hstack((t,dag_label_t.T))
            t = np.hstack((t,bwv_label_t.T))
            t = np.hstack((t,vs_label_t.T))
            t = torch.tensor(t)
         
            #derm
            prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_derm), dim = 1)
        
            prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_derm), dim = 1)

            prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_derm), dim = 1)

            prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_derm), dim = 1)

            prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_derm), dim = 1)

            prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_derm), dim = 1)
    
            prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_derm), dim = 1)

            prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_derm), dim = 1)

            prediction1 = prediction1.unsqueeze(0)
            prediction2 = prediction2.unsqueeze(0)
            prediction3 = prediction3.unsqueeze(0)
            prediction4 = prediction4.unsqueeze(0)
            prediction5 = prediction5.unsqueeze(0)
            prediction6 = prediction6.unsqueeze(0)
            prediction7 = prediction7.unsqueeze(0)
            prediction8 = prediction8.unsqueeze(0)
            prediction1 = np.array(prediction1.cpu(),dtype="int")
            prediction2 = np.array(prediction2.cpu(),dtype="int")
            prediction3 = np.array(prediction3.cpu(),dtype="int")
            prediction4 = np.array(prediction4.cpu(),dtype="int")
            prediction5 = np.array(prediction5.cpu(),dtype="int")
            prediction6 = np.array(prediction6.cpu(),dtype="int")
            prediction7 = np.array(prediction7.cpu(),dtype="int")
            prediction8 = np.array(prediction8.cpu(),dtype="int")
        
            p = np.hstack((prediction1.T,prediction2.T))
            p = np.hstack((p,prediction3.T))
            p = np.hstack((p,prediction4.T))
            p = np.hstack((p,prediction5.T))
            p = np.hstack((p,prediction6.T))
            p = np.hstack((p,prediction7.T))
            p = np.hstack((p,prediction8.T))
            p = torch.tensor(p)
        
            certainty_b = (t == p).long()
            certainty_ones = torch.tensor(np.ones(certainty_b.size()))
            uncertainty_label_derm = []
            for i in range(certainty_ones.size(0)):
                mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
                if mean_squared_error_tmp >=0.6:
                    mean_squared_error_tmp = 1
                else:
                    mean_squared_error_tmp = 0
                uncertainty_label_derm.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
            uncertainty_label_derm = torch.tensor(uncertainty_label_derm)
            #uncertainty
            uncertainty_label_derm = uncertainty_label_derm.float().cuda()
        
            loss_derm = torch.div(
              net.criterion(logit_diagnosis_derm, diagnosis_label)
            + net.criterion(logit_pn_derm, pn_label)
            + net.criterion(logit_str_derm, str_label)
            + net.criterion(logit_pig_derm, pig_label)
            + net.criterion(logit_rs_derm, rs_label)
            + net.criterion(logit_dag_derm, dag_label)
            + net.criterion(logit_bwv_derm, bwv_label)
            + net.criterion(logit_vs_derm, vs_label)
            + net.criterion_MSE(logit_uncertainty_derm,uncertainty_label_derm), 9)

            loss = loss_derm
            
            dia_acc_all = torch.div(net.metric(logit_diagnosis_derm,diagnosis_label),int(derm_image.size(0)))
            sps_acc_all = torch.div(net.metric(logit_pn_derm,pn_label)
                                + net.metric(logit_str_derm,str_label)
                                + net.metric(logit_pig_derm,pig_label)
                                + net.metric(logit_rs_derm,rs_label)
                                + net.metric(logit_dag_derm,dag_label)
                                + net.metric(logit_bwv_derm,bwv_label)
                                + net.metric(logit_vs_derm,vs_label),int(7 * derm_image.size(0)))
            
                                    
        val_loss += loss.item()
        val_dia_acc += dia_acc_all.item()
        vaL_sps_acc += sps_acc_all.item()

    val_loss = val_loss / (index + 1)
    val_dia_acc = val_dia_acc / (index + 1)
    vaL_sps_acc = vaL_sps_acc / (index + 1)

    return val_loss,val_dia_acc,vaL_sps_acc

In [4]:
def run_train(model_name,mode,i):
    log.write('** start Resnet_with_Uncertainty(derm) training here! **\n')
    #best_acc = 0
    es = 0
    patience = 50
    best_mean_acc = 0 
    best_loss = 300
    
    for epoch in range(epochs):
        swa_lr = cosine_learning_schedule[epoch]
        adjust_learning_rate(opt, swa_lr)

        # train_mode
        train_loss,train_dia_acc,train_sps_acc=train(net, train_dataloader,model_name)
        log.write('Round: {}, epoch: {}, Train Loss: {:.4f}, Train Dia Acc: {:.4f}, Train SPS Acc: {:.4f}\n'.format(i, epoch, train_loss,
                                                                                                         train_dia_acc,
                                                                                                         train_sps_acc))

        # validation mode
        val_loss,val_dia_acc,val_sps_acc = validation(net, val_dataloader,model_name)
        
        val_acc = (val_dia_acc + val_sps_acc) / 2
        val_mean_acc = (val_dia_acc*1 + val_sps_acc*7)/8
        
        log.write('Round: {}, epoch: {}, Valid Loss: {:.4f}, Valid Dia Acc: {:.4f}, Valid SPS Acc: {:.4f}\n'.format(i, epoch, val_loss,
                                                                                                         val_dia_acc,
                                                                                                         val_sps_acc))

     
        if val_mean_acc > best_mean_acc:
            es = 0
            best_mean_acc = val_mean_acc
            torch.save(net.state_dict(), out_dir + '/checkpoint/{}_model.pth'.format('best_mean_acc'))
            log.write('Current Best Mean Acc is {}'.format(best_mean_acc))
        #  else:
        #      es += 1
        #      print("Counter {} of {}".format(es,patience))
          
        #      if es > patience:
        #          print("Early stopping with best_mean_acc: {:.4f}".format(best_mean_acc), "and val_mean_acc for this epoch: {:.4f}".format(val_mean_acc))
        #          break
  
  
        if epoch > (epochs - swa_epoch) and epoch % 1 == 0:
            opt.update_swa()
            log.write('SWA Epoch: {}'.format(epoch))

    torch.save(net.state_dict(), out_dir+'/swa_{}_resnet50_model.pth'.format(mode))

In [ ]:
if __name__ == '__main__':
    # Hyperparameters
    mode = 'multimodal'
    tech = "derm"
    model_name = str('Resnet')+str("_")+str(tech)
    shape = (229, 229)
    batch_size = 5
    num_workers = 0
    data_mode = 'Normal'
    deterministic = True
    if deterministic:
        if data_mode == 'Normal':
            random_seeds = 170
        elif data_mode == 'self_evaluated':
            random_seeds = 183
    rounds = 1
    lr = 4.9e-5##
    epochs = 250
    swa_epoch = 150

    train_dataloader, val_dataloader = generate_dataloader(shape, batch_size, num_workers, data_mode,"concat")
    
    for i in range(rounds):
        if deterministic:
            set_seed(random_seeds + i)
      # create logger
        print(random_seeds+i)
        log, out_dir = CreateLogger(mode, model_name,i,data_mode)
        #network 
        net = Resnet_with_uncertainty_derm(class_list).cuda()
        #net = FusionNet(class_list).to(device)
      # create optimizer
        optimizer = optim.Adam(net.parameters(), lr=lr,weight_decay=5e-5)   #weight_decay   #5e-5
        opt = SWA(optimizer)  
      # create learning schedule
        cosine_learning_schedule = create_cosine_learning_schedule(epochs, lr)
        run_train(model_name,mode,i)

170

--- [START IDENTIFIER] ----------------------------------------------------------------


	out_dir      = ./multimodal_Resnet_derm_Normal_weight_file/0/



	<additional comments> ...  

	  - multimodal Resnet_derm  

	  - simple augmentation 



